# Download and Analyze Jira Issues

by Nils Ackermann

## Introduction

Jira offers great reporting capabilities on its GUI. Nevertheless there are certain advanced reporting requirements that cannot be easily fulfilled. Also you might need to feed your Jira data into a data warehouse or other surrounding systems. It is always good to understand how to export your data from your Jira instance to process it elsewhere. 

In this notebook you will learn the following:

- How to establish a secure connection to your personal Jira instance
- How to download issue data
- How to download board and sprint data
- How to download project data
- How to transform Jira data into a [Pandas](https://pandas.pydata.org) dataframe
- How to visualize your Jira data with [Plotly](https://plotly.com/python/basic-charts/)

The notebook uses functionality described in those API:

- https://developer.atlassian.com/cloud/jira/platform/rest/v3/intro/
- https://developer.atlassian.com/cloud/jira/software/rest/api-group-other-operations/
- https://docs.atlassian.com/software/jira/docs/api/REST/7.6.1/

**Remark:** Code has been testes with Jira Cloud.

## 0. Setup Your Python Virtual Environment

It is best to create a separate environment for each notebook / project that you orun. Run the following commands in sequence:

- Create a virtual Python environment in the folder of this notebook:

````
python3 -m venv venv
````

- Activate the virtual environment:

````
source venv/bin/activate
````

- Install all necessary libraries:

````
pip install -r requirements.txt 
````

- Change to the Python Kernel in your IDE that you use to open the Jupyter Notebook like Visual Studio Code

- Create a .env file that will hold your Jira credentials:

````
touch ./.env
````

- The .env file must hold the following environment variables. The file content should look something like this:

````
export JIRA-USER=<your-jira-user-email-address>
export JIRA-API-KEY=<your-jira-api-key>
export JIRA-SERVER=<your-jira-server-url>
````

## 1. Imports and Variables

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from dotenv import load_dotenv
import os

In [ ]:
# Load environment variable holding your secret credentials defined in the .env file
load_dotenv()
user = os.environ.get('JIRA-USER')
apikey = os.environ.get('JIRA-API-KEY')
server = os.environ.get('JIRA-SERVER')

In [ ]:
# Define a few standard colors for the issue types
COLOR_TASK = 'dodgerblue'
COLOR_STORY = 'green'
COLOR_SUBTASK = 'cyan'
COLOR_BUG = 'red'
COLOR_EPIC = 'darkmagenta'

# Mapping of Jira colors to Python colors
COLOR_DICT = {
    'purple':'purple',
    'dark_blue':'darkblue',
    'yellow':'orange',
    'grey':'grey',
    'dark_purple':'purple',
    'blue':'blue',
    'dark_orange':'orange',
    'dark_yellow':'gold',
    'blue-gray':'dodgerblue',
    'green':'green'
}


In [ ]:
# A subset of fields that will be selected from the many fields available in Jira
RELEVANT_FIELDS = ['fields.project.name',
    #'fields.project.key' # This attribute causes issues, unclear why, must investigate
    'fields.project.id',
    'id',
    'key',
    'fields.summary',
    'fields.description',
    'fields.issuetype.name',
    'fields.customfield_10011',
    'fields.customfield_10017',
    'fields.created',
    'fields.status.name',
    'fields.status.statusCategory.key',
    'fields.status.statusCategory.name',
    'fields.status.statusCategory.colorName',
    'fields.priority.name',
    'fields.customfield_10026',
    'fields.creator.displayName',
    'fields.assignee.displayName',
    'fields.reporter.displayName',
    'fields.issuetype.subtask',
    'fields.fixVersions',
    'fields.parent.id',
    'fields.parent.key',
    'fields.parent.fields.summary',
    'fields.parent.fields.status.name',
    'fields.parent.fields.status.statusCategory.key',
    'fields.parent.fields.status.statusCategory.name',
    'fields.parent.fields.status.statusCategory.colorName',
    'fields.parent.fields.priority.name',
    'fields.parent.fields.issuetype.name',
    'fields.parent.fields.issuetype.subtask',
    'fields.aggregatetimespent',
    'fields.resolution',
    'fields.resolutiondate',
    'fields.lastViewed',
    'fields.labels',
    'fields.aggregatetimeoriginalestimate',
    'fields.aggregatetimeestimate',
    'fields.timeestimate',
    'fields.updated',
    'fields.aggregateprogress.progress',
    'fields.aggregateprogress.total',
    'fields.aggregateprogress.percent',
    'fields.progress.percent',
    'fields.timeoriginalestimate',
    'fields.duedate',
    'fields.progress.progress',
    'fields.progress.total',
    'fields.resolution.name',
    'fields.statuscategorychangedate',
    'fields.status.statusCategory.colorName']

In [ ]:
# Dictionary to change the standard Jira field name to our own wording
COLUMN_NAME_DICT = {
    'fields.project.name': 'project_name',
    #'fields.project.key': 'project_key', # This attribute causes issues, unclear why, must investigate
    'fields.project.id': 'project_id',
    'id': 'issue_id',
    'key': 'issue_key',
    'fields.summary': 'summary', 
    'fields.description': 'description', 
    'fields.issuetype.name': 'issuetype', 
    'fields.customfield_10011': 'epic_name', 
    'fields.customfield_10017': 'epic_color', 
    'fields.created': 'created', 
    'fields.status.name': 'status', 
    'fields.status.statusCategory.key': 'status_cat_key', 
    'fields.status.statusCategory.name': 'status_cat_name', 
    'fields.status.statusCategory.colorName': 'status_cat_color', 
    'fields.priority.name': 'prio', 
    'fields.customfield_10026': 'storypoints', 
    'fields.creator.displayName': 'creator', 
    'fields.assignee.displayName': 'assignee', 
    'fields.reporter.displayName': 'reporter', 
    'fields.issuetype.subtask': 'is_subtask', 
    'fields.fixVersions': 'fix_versions', 
    'fields.parent.id': 'parent_id', 
    'fields.parent.key': 'parent_key', 
    'fields.parent.fields.summary': 'parent_summary', 
    'fields.parent.fields.status.name': 'parent_status', 
    'fields.parent.fields.status.statusCategory.key': 'parent_status_cat_key', 
    'fields.parent.fields.status.statusCategory.name': 'parent_status_cat', 
    'fields.parent.fields.status.statusCategory.colorName': 'parent_status_cat_color', 
    'fields.parent.fields.priority.name': 'parent_prio', 
    'fields.parent.fields.issuetype.name': 'parent_issuetype', 
    'fields.parent.fields.issuetype.subtask': 'parent_is_subtask', 
    'fields.aggregatetimespent': 'aggregatetimespent', 
    'fields.resolution': 'resolution', 
    'fields.resolutiondate': 'resolution_date', 
    'fields.lastViewed': 'last_viewed', 
    'fields.labels': 'labels', 
    'fields.aggregatetimeoriginalestimate': 'aggregatetimeoriginalestimate', 
    'fields.aggregatetimeestimate': 'aggregatetimeestimate', 
    'fields.timeestimate': 'timeestimate', 
    'fields.updated': 'updated', 
    'fields.aggregateprogress.progress': 'aggregateprogress', 
    'fields.aggregateprogress.total': 'aggregateprogress_total', 
    'fields.aggregateprogress.percent': 'aggregateprogress_percent', 
    'fields.progress.percent': 'progress_percent', 
    'fields.timeoriginalestimate': 'timeoriginalestimate', 
    'fields.duedate': 'duedate', 
    'fields.progress.progress': 'progress', 
    'fields.progress.total': 'progress_total', 
    'fields.resolution.name': 'resolution', 
    'fields.statuscategorychangedate': 'status_cat_changedate', 
    'fields.status.statusCategory.colorName': 'status_cat_color' 
}

## 2. Helper Functions

Various functions that are used throughtout the notebook that connect to the Jira server and download data. All functions return the data as a Pandas dataframe. 

In [ ]:
def jira_auth_get_connect(url_path, *args, **kwargs):

    # Version: 1.00
    # Last Updated: Feb-11-2023

    # Establishes an authenticated connection to Jira API with the given URL
    # Parameters must be added as variable param which should include a dictionary
    # (example: params = {'jql':'project=abc','maxResults':80})

    import base64
    import requests
    import json
    from urllib.error import HTTPError
    body = ''
    if kwargs.get('body', None) is not None:
            body = kwargs.get('body')
            body = json.dumps(body)
    url = server+url_path
    auth_string = user + ":" + apikey
    signature = base64.b64encode(bytes(auth_string, "utf-8")).decode("utf-8")
    headers = {
        "Accept":"application/json",
        "Content-Type":"application/json",
        "Authorization":"Basic " + signature
    }
    try:
        if kwargs.get('param', None) is not None:
            params = kwargs.get('param')
            response = requests.get(url, params, headers=headers)
        else:
            response = requests.get(url, headers=headers)
        response.raise_for_status()
        print(f'HTTP connection {url} successful!')
        return response
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return http_err
    except Exception as err:
        print(f'Other error occurred: {err}')
        return err

In [ ]:
def jira_get_issues(jql = '') -> pd.DataFrame:

    # Version: 1.00
    # Last Updated: Feb-11-2023

    # Gets all issues for the given query. Jira REST API will be called multiple times
    # until all issues are fetched. Note: maximum block size is 100. Returns a Pandas dataframe.

    import json
    all_issues = []
    block_size = 100
    start_position = 0
    has_next = True
    while has_next:
        #params = {'jql':jql, 'startAt':start_position, 'maxResults':block_size}
        params = {'startAt':start_position, 'maxResults':block_size}
        if jql != '':
            params['jql'] = jql
        response = jira_auth_get_connect('/rest/api/latest/search',param=params)
        #print(json.dumps(json.loads(response.text)['issues'], sort_keys=True, indent=4, separators=(",", ": ")))
        issues_in_block = json.loads(response.text)['issues']
        all_issues.extend(issues_in_block)
        if len(issues_in_block) == 0:
            has_next = False
        start_position += block_size
    df_issues = pd.json_normalize(all_issues)
    return df_issues

In [ ]:
def jira_get_all_projects() -> pd.DataFrame:

    # Version: 1.00
    # Last Updated: Feb-11-2023

    # Gets all projects. Jira REST API will be called multiple times
    # until all projects are fetched. Note: maximum block size is 100.
    # Returns a Pandas dataframe.

    import json
    all_projects = []
    block_size = 100
    start_position = 0
    has_next = True
    while has_next:
        params = {'startAt':start_position, 'maxResults':block_size}
        response = jira_auth_get_connect('/rest/api/latest/project/search',param=params)
        #print(json.dumps(json.loads(response.text)['values'], sort_keys=True, indent=4, separators=(",", ": ")))
        projects_in_block = json.loads(response.text)['values']
        all_projects.extend(projects_in_block)
        if len(projects_in_block) == 0:
            has_next = False
        start_position += block_size
    df_projects = pd.json_normalize(all_projects)
    return df_projects

In [ ]:
def jira_get_versions(project_key) -> pd.DataFrame:

    # Version: 1.00
    # Last Updated: Feb-11-2023

    # Gets all versions for the project key. Jira REST API will be called multiple times
    # until all versions are fetched. Returns a Pandas dataframe.

    import json
    response = jira_auth_get_connect('/rest/api/latest/project/'+project_key)
    #print(json.dumps(json.loads(response.text)['versions'], sort_keys=True, indent=4, separators=(",", ": ")))
    all_versions = json.loads(response.text)['versions']
    df_versions = pd.json_normalize(all_versions)
    return df_versions

In [ ]:
def jira_get_boards(project_key = '') -> pd.DataFrame:

    # Version: 1.1
    # Last Updated: April-02-2023

    # Gets all boards. Jira REST API will be called multiple times
    # until all boards are fetched. Note: maximum block size is 100.
    # Returns a Pandas dataframe.

    import json
    all_boards = []
    block_size = 100
    start_position = 0
    has_next = True
    while has_next:
        params = {'startAt':start_position, 'maxResults':block_size}
        if project_key != '':
            params['projectKeyOrId'] = project_key
        response = jira_auth_get_connect('/rest/agile/1.0/board',param=params)
        #print(json.dumps(json.loads(response.text)['values'], sort_keys=True, indent=4, separators=(",", ": ")))
        boards_in_block = json.loads(response.text)['values']
        all_boards.extend(boards_in_block)
        if len(boards_in_block) == 0:
            has_next = False
        start_position += block_size
    df_boards = pd.json_normalize(all_boards)
    return df_boards

In [ ]:
def jira_get_sprints(board_id) -> pd.DataFrame:

    # Version: 1.00
    # Last Updated: Feb-11-2023

    # Gets all sprints for the given board id. Jira REST API will be called multiple times
    # until all sprints are fetched. Note: maximum block size is 100.
    # Returns a Pandas dataframe.

    import requests
    import json
    all_sprints = []
    block_size = 100
    start_position = 0
    has_next = True
    while has_next:
        params = {'startAt':start_position, 'maxResults':block_size}
        response = jira_auth_get_connect(f'/rest/agile/1.0/board/{board_id}/sprint',param=params)
        if type(response) is not requests.exceptions.HTTPError:
            sprints_in_block = json.loads(response.text)['values']
            all_sprints.extend(sprints_in_block)
            if len(sprints_in_block) == 0:
                has_next = False
            start_position += block_size
        else:
            break
    df_sprints = pd.json_normalize(all_sprints)
    return df_sprints

In [ ]:
def jira_get_backlog_issues(board_id) -> pd.DataFrame:
    
    # Version: 1.00
    # Last Updated: Feb-11-2023

    # Gets all issues from backlog for the given board id. Jira REST API will be called multiple times
    # until all issues of backlog are fetched. Note: maximum block size is 100.
    # Returns a Pandas dataframe.  

    import requests
    all_backlog_issues = []
    block_size = 100
    start_position = 0
    has_next = True
    while has_next:
        params = {'startAt':start_position, 'maxResults':block_size}
        response = jira_auth_get_connect(f'/rest/agile/1.0/board/{board_id}/backlog',param=params)
        if type(response) is not requests.exceptions.HTTPError:
            backlogs_in_block = json.loads(response.text)['issues']
            all_backlog_issues.extend(backlogs_in_block)
            if len(backlogs_in_block) == 0:
                has_next = False
            start_position += block_size
        else:
            break
    df_backlog_issues = pd.json_normalize(all_backlog_issues)
    return df_backlog_issues

In [ ]:
def jira_get_sprint_issues(sprint_id) -> pd.DataFrame:

    # Version: 1.00
    # Last Updated: Feb-11-2023

    # Gets all issues from sprint for the given sprint id. Jira REST API will be called multiple times
    # until all issues of sprint are fetched. Note: maximum block size is 100.
    # Returns a Pandas dataframe.  

    import requests
    import json
    all_issues = []
    block_size = 100
    start_position = 0
    has_next = True
    while has_next:
        params = {'startAt':start_position, 'maxResults':block_size}
        response = jira_auth_get_connect(f'/rest/agile/1.0/sprint/{sprint_id}/issue',param=params)
        if type(response) is not requests.exceptions.HTTPError:
            issues_in_block = json.loads(response.text)['issues']
            all_issues.extend(issues_in_block)
            if len(issues_in_block) == 0:
                has_next = False
            start_position += block_size
        else:
            break
    df_issues = pd.json_normalize(all_issues)
    return df_issues

In [ ]:
def jira_get_board_issues(board_id) -> pd.DataFrame:

    # Version: 1.00
    # Last Updated: Feb-11-2023

    # Gets basic data for all issues including their sprint reference 
    # for the given board id. Returns a Pandas dataframe.  

    df_all_sprints = jira_get_sprints(board_id = board_id)
    sprint_ids = df_all_sprints.id.to_list()
    issue_rows = []
    for sprint_id in sprint_ids:
        df_issues = jira_get_sprint_issues(sprint_id=sprint_id)[['id','key','fields.project.name','fields.project.id','fields.issuetype.name','fields.status.name','fields.status.statusCategory.name','fields.priority.name','fields.customfield_10026']]
        df_issues['sprint']=sprint_id
        issues = df_issues.values.tolist()
        issue_rows.extend(issues)
    df_board_issues = pd.DataFrame(issue_rows)
    df_board_issues.columns = ['issue_id','issue_key','project_name','project_id','issuetype','status','status_cat_name','prio','storypoints','sprint_id']
    df_board_issues_enhanced = pd.merge(df_board_issues, df_all_sprints, how='left', left_on='sprint_id', right_on = 'id')
    df_board_issues_enhanced.drop(['id','self','goal'], axis=1, inplace=True)
    df_board_issues_enhanced.rename(columns = {'name':'sprint_name','state':'sprint_state','startDate':'sprint_start_date','endDate':'sprint_end_date','completeDate':'sprint_complete_date','originBoardId':'board_id'}, inplace = True)
    return df_board_issues_enhanced

## 3. Jira Issues

Issues are at the core of Jira. Therefore we will start with the download of all issues. 

### 3.1 Get a Specific Issue

The simplest case to retrieve one particular issue based on the issue key.

In [ ]:
response = jira_auth_get_connect('/rest/agile/1.0/issue/BEAS-298')
df_issue = pd.json_normalize(json.loads(response.text))

### 3.2 Get Multiple Issues

Get issues up to maxResults 100:

In [ ]:
#params = {'jql':'project = abc','maxResults':80}
#response = jira_auth_get_connect('/rest/api/3/search',param=params)
#print(json.dumps(json.loads(response.text)['issues'], sort_keys=True, indent=4, separators=(",", ": ")))
#ll_issues = json.loads(response.text)['issues']
#df_issues = pd.json_normalize(all_issues)

### 3.3 Get All Issues for JQL Query

Going forward, we will use the custom functions defined above. All functions are written in a way that they retrieve all data in batches. The first query retrieves all issues that match the given jql query. If the query is left blank, all issues available in your Jira instance are returned. 

In [ ]:
#df_issues = jira_get_issues('project=abc')
df_issues = jira_get_issues()

In the following code block, we will enhance the raw Jira issue data as following:

- Only a subset of fields is retained (all fields in variable RELEVANT_FIELDS)
- The selected fields are renamed (in line with COLUMN_NAME_DICT dictionary)
- The issue data is merged with the epic data
- Jira colors are replaced with more appropriate colors

The resulting dataframe df_issues_enhanced is used in the subsequent code snippets of this Jupyter Notebook.

In [ ]:
df_issues_enhanced = df_issues[df_issues.columns[df_issues.columns.isin(RELEVANT_FIELDS)]]
df_issues_enhanced.rename(columns=COLUMN_NAME_DICT, inplace=True)
# Adding the associated epic name and color to every issue (if available)
df_epics = df_issues_enhanced[['issue_id','epic_name','epic_color']]
df_issues_enhanced = pd.merge(df_issues_enhanced, df_epics, how='left', left_on='parent_id', right_on = 'issue_id')
df_issues_enhanced.loc[df_issues_enhanced['epic_name_x'].isnull(),'epic_name_x'] = df_issues_enhanced['epic_name_y']
df_issues_enhanced.loc[df_issues_enhanced['epic_color_x'].isnull(),'epic_color_x'] = df_issues_enhanced['epic_color_y']
df_issues_enhanced.drop(['issue_id_y','epic_name_y','epic_color_y'], axis=1, inplace=True)
df_issues_enhanced.rename(columns = {'issue_id_x':'issue_id','epic_name_x':'epic_name','epic_color_x':'epic_color'}, inplace = True)
df_issues_enhanced.epic_name = df_issues_enhanced.epic_name.fillna('No Epic')
# Replace colors
df_issues_enhanced['epic_color'] = df_issues_enhanced['epic_color'].apply(lambda x: COLOR_DICT.get(x,'white'))
df_issues_enhanced['status_cat_color'] = df_issues_enhanced['status_cat_color'].apply(lambda x: COLOR_DICT.get(x,'white'))
df_issues_enhanced['parent_status_cat_color'] = df_issues_enhanced['parent_status_cat_color'].apply(lambda x: COLOR_DICT.get(x,'white'))

## 4 Basic Visualization of Issues

In [ ]:
# Extract colors for status category
STATUS_CAT_COLOR_DICT = df_issues_enhanced[['status_cat_name','status_cat_color']].drop_duplicates().set_index('status_cat_name')['status_cat_color'].to_dict()

### 4.1 Show Storypoints Grouped by Issue Type

**Purpose:** As a program manager I want to see the total volume of work (in story points) per type or work (e.g. tasks, bug-fixing) so that I can better understand where the teams spent the majority of their time.

In [ ]:
# Query
df_issues_per_type = df_issues_enhanced.query("issuetype != 'Epic'")[['issuetype','status_cat_name','storypoints','prio']].groupby(['issuetype','status_cat_name','prio'])['storypoints'].agg(storypoints='sum', issue_count='count').reset_index()

In [ ]:
# histogram instead of bar to merge the priority values into one color
fig = px.histogram(df_issues_per_type, x="issuetype", y="storypoints", color='status_cat_name',
    title='Storypoints per Issue Type',
    color_discrete_map= STATUS_CAT_COLOR_DICT,
    labels={'status_cat_name':'Status','storypoints':'Storypoints','issue_count':'Number of Issues','issuetype':'Issue Type'})
fig.update_layout(
    yaxis_title="Storypoints",
)
fig.show()

### 4.2 Show Number of Issues Grouped by Issue Type

**Purpose:** As a program manager I want to see the total volume of generated work per type (e.g. tasks, bug-fixing) so that I can better understand the volume of work for each work type.

In [ ]:
fig = px.pie(df_issues_per_type,
    values='issue_count',
    names='issuetype',
    color='issuetype',
    title='Number of Issues per Issue Type',
    labels={'issue_count':'Number of Issues','issuetype':'Issue Type'},
    color_discrete_map={
        "Story": COLOR_STORY,
        "Bug": COLOR_BUG,
        "Task": COLOR_TASK,
        "Sub-task": COLOR_SUBTASK})
fig.update_traces(textinfo="percent+label")
fig.update_layout(showlegend=False)
fig.show()

### 4.3 Show Volume of Work per Epic

**Purpose:** As a program manager I want to see the total volume of work (in story points and by ticket volume) for each Epic so that I can better understand on which features we spent most of our time. 

In [ ]:
# Query
df_issues_per_epic = df_issues_enhanced.query("issuetype != 'Epic'")[['issuetype','epic_name','status_cat_name','storypoints','prio']].groupby(['issuetype','epic_name','status_cat_name','prio'])['storypoints'].agg(storypoints='sum', issue_count='count').reset_index()

In [ ]:
fig = px.treemap(df_issues_per_epic, path=[px.Constant("All Work"), 'epic_name', 'issuetype', 'prio'],
    values='storypoints',
    color='epic_name',
    title='Epic Treemap (Based on Storypoints)',
    labels={'storypoints':'Storypoints','labels':'Labels','parent':'Parent','id':'ID','epic_name':'Epic'})
fig.update_traces(root_color="white")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
fig = px.treemap(df_issues_per_epic, path=[px.Constant("All Work"), 'epic_name', 'issuetype', 'prio'],
    values='issue_count',
    color='epic_name',
    title='Epic Treemap (Based on Issue Count)',
    labels={'issue_count':'Issue Count','labels':'Labels','parent':'Parent','id':'ID','epic_name':'Epic'})
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

### 4.4 Show Current Progress per Project

**Purpose:** As a program manager I want to see the progress for each project by comparing story points for ongoing and already completed issues so that I can understand how much work is still ahead of us.

In [ ]:
# Query
df_issues_per_project = df_issues_enhanced.query("issuetype != 'Epic'")[['project_name','issuetype','status_cat_name','storypoints']].groupby(['project_name','issuetype','status_cat_name'])['storypoints'].agg(storypoints='sum', issue_count='count').reset_index()

In [ ]:
fig = px.histogram(df_issues_per_project,
    y="project_name",
    x="storypoints",
    orientation='h',
    color='status_cat_name',
    title='Progress per Project',
    color_discrete_map= STATUS_CAT_COLOR_DICT,
    labels={'project_name':'Project','status_cat_name':'Status','storypoints':'Storypoints','issue_count':'Number of Issues','issuetype':'Issue Type'})
fig.update_layout(
    yaxis_title="Projects",
    xaxis_title="Storypoints",
)
fig.show()

## 5. Jira Boards and Sprints

### 5.1 Get All Available Boards

In [ ]:
df_boards = jira_get_boards()
df_boards[['location.projectId','location.projectKey','location.projectName','id','name','type','location.displayName','location.name']].head(10)

### 5.2 Get All Sprints for a Specific Board

In [ ]:
#df_sprints = jira_get_sprints(board_id=7)
#df_sprints[['id','state','name','startDate','endDate','completeDate','originBoardId','goal']]

### 5.3 Get All Issues From a Board Backlog

In [ ]:
#df_backlog_issues = jira_get_backlog_issues(board_id=7)

### 5.4 Get All Issues From a Particular Sprint

In [ ]:
#df_sprint_issues = jira_get_sprint_issues(sprint_id=32)

### 5.5 Get All Issues From Board Including Sprint Reference

**Attention:** Query takes long since it must make dedicated HTTP calls for every sprint in the board

In [ ]:
df_issues_in_sprints = jira_get_board_issues(board_id=7)

### 5.6 Visualize Sprint Progress

Similar to the Jira sprint velocity report, we will generate a similar chart. 

In [ ]:
# Query
df_issues_per_sprint = df_issues_in_sprints.query("project_name == 'abc'")[['project_name','issuetype','prio','storypoints','sprint_name','sprint_start_date','sprint_end_date']].groupby(['project_name','issuetype','prio','sprint_name','sprint_start_date','sprint_end_date'])['storypoints'].agg(storypoints='sum').reset_index()

In [ ]:
# histogram instead of bar to merge values into one color
fig = px.histogram(df_issues_per_sprint, x="sprint_name", y="storypoints", color='issuetype',
    title='Storypoints per Issue Type',
    labels={'sprint_name':'Sprint Name','issue_count':'Number of Issues','issuetype':'Issue Type'},
    color_discrete_map={
        "Story": COLOR_STORY,
        "Bug": COLOR_BUG,
        "Task": COLOR_TASK,
        "Sub-task": COLOR_SUBTASK})
fig.update_layout(
    yaxis_title="Storypoints",
    xaxis_title="Sprints",
)
fig.show()

## 6. Jira Projects

### 6.1 Get all Jira Projects

In [ ]:
df_projects = jira_get_all_projects()
df_projects[['id','key','name','projectTypeKey','simplified','style']]

### 6.2 Get All Versions for One Project

In [ ]:
df_versions = jira_get_versions('BEAS')
df_versions[['projectId','id','description','name','archived','released','startDate','releaseDate','userStartDate','userReleaseDate']].head(10)